In [1]:
from keras.models import Model,Sequential
from keras.layers import Dense, Flatten
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.utils import multi_gpu_model
from keras.losses import categorical_crossentropy
from keras.optimizers import rmsprop,adam
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import numpy as np
import cv2
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import os
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
from moxing import file as mfile

INFO:root:Using MoXing-v1.14.0-3c8d0e90
INFO:root:Using OBS-Python-SDK-3.1.2


In [4]:
mfile.copy('s3://lincc/lincc/origin.tar.gz', './origin.tar.gz')

In [5]:
!tar -xzvf origin.tar.gz

origin/
origin/01562828258.8049057.5.tiff
origin/10001562828258.6803238.8.tiff
origin/10091562828257.9266012.8.tiff
origin/10101562828257.8652837.1.tiff
origin/10131562828258.6860137.7.tiff
origin/101562828258.1909595.16.tiff
origin/10181562828258.2658179.18.tiff
origin/10191562828257.775537.4.tiff
origin/10261562828258.424716.20.tiff
origin/10371562828258.6763663.17.tiff
origin/10421562828258.623299.18.tiff
origin/10461562828257.808075.6.tiff
origin/10491562828258.6842604.4.tiff
origin/10531562828258.4374785.20.tiff
origin/10571562828258.672421.6.tiff
origin/10601562828258.5002568.22.tiff
origin/10621562828258.6529515.6.tiff
origin/10621562828258.6750536.16.tiff
origin/10651562828258.1081126.21.tiff
origin/10701562828257.7639692.0.tiff
origin/10761562828258.4776962.15.tiff
origin/10761562828258.8531532.14.tiff
origin/10801562828258.8385777.2.tiff
origin/10901562828258.6984208.13.tiff
origin/10971562828258.2449822.14.tiff
origin/1111562828258.4317741.14.tiff
origin/11131562828258.64633

origin/25571562828258.5615518.10.tiff
origin/25591562828258.380146.16.tiff
origin/25621562828257.780511.10.tiff
origin/25621562828258.663064.14.tiff
origin/25631562828258.6184635.18.tiff
origin/25671562828258.1644874.17.tiff
origin/25691562828257.7925777.19.tiff
origin/25691562828258.3139641.1.tiff
origin/25701562828258.659545.0.tiff
origin/25741562828257.77664.2.tiff
origin/25751562828258.1676185.16.tiff
origin/25781562828258.206433.11.tiff
origin/2581562828258.2980826.19.tiff
origin/2581562828258.340921.13.tiff
origin/25871562828258.0260437.19.tiff
origin/25871562828258.0601652.17.tiff
origin/25911562828258.484747.19.tiff
origin/25941562828258.8819034.4.tiff
origin/25971562828258.2427504.7.tiff
origin/26001562828258.6003728.3.tiff
origin/2601562828258.3351834.6.tiff
origin/2601562828258.5553277.22.tiff
origin/26071562828258.6118138.3.tiff
origin/26091562828258.2843904.4.tiff
origin/26161562828258.771379.11.tiff
origin/26221562828258.7315967.12.tiff
origin/26331562828258.3042634.6.tif

origin/33721562828257.8465362.11.tiff
origin/33731562828258.4194586.21.tiff
origin/33791562828257.9028127.9.tiff
origin/3381562828258.7594457.4.tiff
origin/33821562828258.4657533.16.tiff
origin/33831562828257.800886.8.tiff
origin/33881562828258.5977395.6.tiff
origin/33891562828257.8997746.5.tiff
origin/33911562828257.7760947.8.tiff
origin/33941562828258.7448206.21.tiff
origin/34081562828258.4357302.0.tiff
origin/34101562828257.9009955.21.tiff
origin/34111562828257.962759.3.tiff
origin/34181562828258.369552.11.tiff
origin/34201562828257.9614508.14.tiff
origin/34211562828258.1027775.13.tiff
origin/34211562828258.8805816.9.tiff
origin/34221562828258.5407774.8.tiff
origin/34241562828258.2914362.15.tiff
origin/34301562828258.5482967.7.tiff
origin/34321562828257.7943456.22.tiff
origin/34321562828258.3656054.22.tiff
origin/34351562828258.3629725.14.tiff
origin/34381562828257.7978885.13.tiff
origin/34481562828258.7828727.10.tiff
origin/34501562828258.4909074.18.tiff
origin/3451562828258.492685

origin/48661562828258.2923415.21.tiff
origin/48691562828258.7545922.18.tiff
origin/48721562828258.8447793.16.tiff
origin/48761562828258.539906.17.tiff
origin/48891562828258.2645013.5.tiff
origin/48931562828258.175157.3.tiff
origin/48931562828258.4768202.7.tiff
origin/48941562828258.8610897.17.tiff
origin/48971562828258.297636.20.tiff
origin/4901562828257.7361012.4.tiff
origin/49091562828258.276039.10.tiff
origin/49111562828258.5597837.6.tiff
origin/49121562828258.8421414.18.tiff
origin/49141562828258.5981843.7.tiff
origin/49231562828258.1724806.7.tiff
origin/49281562828258.202465.14.tiff
origin/49291562828258.45117.1.tiff
origin/49321562828257.7464197.7.tiff
origin/49321562828258.7647574.17.tiff
origin/49381562828258.5738878.15.tiff
origin/49451562828257.8955333.10.tiff
origin/49521562828257.8068824.12.tiff
origin/49521562828258.4256217.20.tiff
origin/49531562828257.7890322.14.tiff
origin/49611562828258.8120604.5.tiff
origin/49651562828258.0172796.16.tiff
origin/49651562828258.6944113.

origin/56931562828257.853752.7.tiff
origin/56961562828257.7355611.8.tiff
origin/5701562828258.6728628.2.tiff
origin/57071562828257.7220812.9.tiff
origin/57111562828257.9634156.7.tiff
origin/57201562828258.2941194.8.tiff
origin/57261562828257.9489634.17.tiff
origin/5731562828258.2680507.15.tiff
origin/57321562828257.903419.18.tiff
origin/57381562828258.783749.8.tiff
origin/57391562828258.203345.20.tiff
origin/57431562828258.7192.15.tiff
origin/57471562828258.3069274.5.tiff
origin/57521562828257.9706776.19.tiff
origin/57581562828258.4238484.13.tiff
origin/57661562828258.663523.15.tiff
origin/57771562828257.8026695.3.tiff
origin/57831562828258.0548081.10.tiff
origin/57831562828258.3440015.5.tiff
origin/58041562828257.7469642.7.tiff
origin/58101562828258.8628635.18.tiff
origin/58111562828258.882345.19.tiff
origin/5811562828258.2347748.14.tiff
origin/5811562828258.8027074.14.tiff
origin/58121562828257.7645135.13.tiff
origin/58151562828257.9825177.16.tiff
origin/58201562828257.756308.19.tiff

origin/7431562828258.8138108.11.tiff
origin/74341562828257.7884395.19.tiff
origin/74441562828258.2405465.3.tiff
origin/74501562828258.2051184.12.tiff
origin/74581562828258.2046654.5.tiff
origin/74621562828258.5447702.18.tiff
origin/74651562828258.6807575.8.tiff
origin/74761562828258.775774.19.tiff
origin/74771562828258.088797.15.tiff
origin/74801562828258.622863.8.tiff
origin/7481562828257.937117.7.tiff
origin/74851562828258.7151918.18.tiff
origin/74861562828258.7090316.11.tiff
origin/74891562828258.4904566.12.tiff
origin/74901562828258.4825552.18.tiff
origin/74991562828258.2374387.12.tiff
origin/7501562828258.235663.0.tiff
origin/75041562828258.4203408.10.tiff
origin/75121562828257.8429208.6.tiff
origin/75171562828258.6776996.10.tiff
origin/75231562828258.113431.1.tiff
origin/75251562828257.9515774.2.tiff
origin/75281562828258.2454193.13.tiff
origin/75441562828258.5540056.9.tiff
origin/75551562828258.4759464.9.tiff
origin/7581562828258.885862.7.tiff
origin/75841562828258.5504904.18.ti

origin/83471562828258.781484.4.tiff
origin/83551562828258.4838748.15.tiff
origin/83611562828257.8761544.4.tiff
origin/83711562828257.9364557.3.tiff
origin/83781562828257.718611.19.tiff
origin/8381562828258.4675221.14.tiff
origin/83851562828257.9904804.4.tiff
origin/83901562828258.6754973.11.tiff
origin/83931562828258.6325827.21.tiff
origin/83981562828257.9535549.21.tiff
origin/83981562828258.24813.19.tiff
origin/84031562828258.6295075.11.tiff
origin/84071562828258.874347.20.tiff
origin/84111562828257.8230326.18.tiff
origin/84251562828258.4087534.4.tiff
origin/84261562828257.966702.22.tiff
origin/84281562828258.3431296.2.tiff
origin/84291562828258.3108597.2.tiff
origin/84301562828258.461353.20.tiff
origin/84311562828257.74093.7.tiff
origin/84321562828258.5217896.0.tiff
origin/84321562828258.5328805.22.tiff
origin/84331562828258.4997659.3.tiff
origin/84361562828258.7660818.14.tiff
origin/84421562828258.143636.14.tiff
origin/84421562828258.6136236.10.tiff
origin/84421562828258.8270917.1.t

origin/91721562828258.67682.22.tiff
origin/91811562828258.3576205.2.tiff
origin/9181562828258.6202283.4.tiff
origin/91831562828258.1555634.2.tiff
origin/91871562828258.3448796.2.tiff
origin/91871562828258.8310614.2.tiff
origin/91931562828258.7099113.17.tiff
origin/91961562828258.2059915.22.tiff
origin/91981562828258.0474584.9.tiff
origin/92041562828258.7969942.20.tiff
origin/92061562828258.863736.12.tiff
origin/92071562828257.9160793.11.tiff
origin/92091562828258.0708194.20.tiff
origin/9211562828258.616691.20.tiff
origin/92151562828258.3977523.14.tiff
origin/92231562828258.8832257.20.tiff
origin/92331562828258.364294.16.tiff
origin/92451562828258.5231123.14.tiff
origin/92501562828258.797434.2.tiff
origin/92551562828257.9052534.12.tiff
origin/92571562828257.9765947.9.tiff
origin/92581562828258.7461498.19.tiff
origin/92601562828258.8319373.4.tiff
origin/92631562828258.2312403.20.tiff
origin/92661562828258.719637.14.tiff
origin/92691562828257.7844174.2.tiff
origin/92771562828258.6065214.0

origin/3831562828259.1334167.2.tiff
origin/38781562828259.2154217.2.tiff
origin/38801562828259.2412832.20.tiff
origin/38821562828258.9548464.1.tiff
origin/39611562828258.9394145.8.tiff
origin/39681562828259.2669542.18.tiff
origin/39821562828258.9552786.14.tiff
origin/39951562828259.2660613.10.tiff
origin/40031562828259.2933168.17.tiff
origin/40051562828259.2991865.15.tiff
origin/40161562828259.2502785.6.tiff
origin/40361562828259.2597244.9.tiff
origin/40641562828259.018821.16.tiff
origin/40991562828259.0941916.14.tiff
origin/41191562828259.2556975.9.tiff
origin/41201562828259.1876702.22.tiff
origin/41301562828259.1804821.14.tiff
origin/41421562828259.2818456.12.tiff
origin/41691562828259.1974638.21.tiff
origin/41831562828259.2892275.17.tiff
origin/421562828258.967525.21.tiff
origin/42191562828259.32648.14.tiff
origin/42291562828258.9690697.21.tiff
origin/42671562828258.9634538.5.tiff
origin/42811562828259.169403.8.tiff
origin/42881562828259.0476427.21.tiff
origin/43061562828258.9763598

In [12]:
num_classes = 24
batch_size = 100
early_stop_patience = 15
epochs = 100
lr = 0.0005
image_size = 224
split_symbol = '/'  # linux or mac uses '/' windows uses '\\'

In [4]:
def inception_residual_network(input_shape):
    #model = Sequential()
    inception = InceptionResNetV2(include_top=False,
                                  input_shape=input_shape)
    #model.add(inception.output)
    #model.add(Flatten(name='flatten'))
    model = Flatten(name='flatten')(inception.output)
    #model.add(Dense(num_classesa, activation='softmax'))
    model = Dense(num_classes, activation='softmax')(model)
    model = Model(inception.input, model, name='inception_residual_network')
    #model.summary()
    return model

In [8]:
model = inception_residual_network((image_size, image_size, 3))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [14]:
def training(model, x_train, y_train, x_test, y_test, model_name):
    model.compile(loss=categorical_crossentropy,
                  optimizer=adam(lr=lr),
                  #optimizer=rmsprop(lr=lr, decay=0.9),
                  metrics=['accuracy'])
    early_stop_callback = EarlyStopping(monitor='val_acc',
                                        patience=early_stop_patience,
                                        verbose=1,
                                        restore_best_weights=True,
                                        mode='auto')
    model.fit(x_train, y_train,
                       batch_size=batch_size,
                       epochs=epochs,
                       verbose=1,
                       callbacks=[early_stop_callback],
                       validation_data=(x_test, y_test))
    test_score = model.evaluate(x_test, y_test, verbose=1)
    print(test_score)
    val_acc = test_score[1]
    model.save(u'model_name' + '.' + str(val_acc) + '.h5')
    return model

In [6]:
def get_origin_data():
    X = []
    Y = []
    path = u'./origin/'
    for dir_path, dir_names, file_names in os.walk(path):
        for file_name in file_names:
            file_path = dir_path + split_symbol + file_name
            array = np.array(Image.open(file_path), dtype=np.uint8)
            X.append(array)
            label_y = int(file_name.split('.')[-2])
            Y.append(label_y)
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [7]:
X, Y = get_origin_data()
print('X.shape, Y.shape:', X.shape, Y.shape)

X.shape, Y.shape: (2986, 224, 224) (2986,)


In [8]:
scores = []
index = 0
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [15]:
for train_idx, val_idx in kfold.split(X, Y):
    train_data = X[train_idx]  # 80% of data
    val_data = X[val_idx]  # 20% of data
    train_target = Y[train_idx]
    val_target = Y[val_idx]

    # augmentation of train_data
    train_x, train_y = [], []
    for (x, y) in zip(train_data, train_target):
        label_y = np.zeros(24)
        label_y[y] = 1
        #for times in range(int(360 / 15)):
        train_y.append(label_y)
        train_x.append(x)
        #train_x.append(np.array(Image.fromarray(x).rotate(times * 15)))
    train_x = np.array(train_x)
    rain_x = train_x.reshape(train_x.shape[0], train_x.shape[1], train_x.shape[2], 1)
    train_y = np.array(train_y)
    train_x, train_y = shuffle(train_x, train_y)

        # prepare val data
    val_x, val_y = [], []
    for (x, y) in zip(val_data, val_target):
        label_y = np.zeros(24)
        label_y[y] = 1
        val_y.append(label_y)
        val_x.append(x)
        #val_x.append(np.array(Image.fromarray(x).rotate(random.randint(0, 360))))
    val_x = np.array(val_x)
    val_x = val_x.reshape(val_x.shape[0], val_x.shape[1], val_x.shape[2], 1)

    train_x = [cv2.cvtColor(i, cv2.COLOR_GRAY2BGR) for i in train_x]
    train_x = np.array(train_x)
    val_x = [cv2.cvtColor(i, cv2.COLOR_GRAY2BGR) for i in val_x]
    val_x = np.array(val_x)
    val_y = np.array(val_y)
    val_x, val_y = shuffle(val_x, val_y)

    model = inception_residual_network((image_size, image_size, 3))
    model_name = 'inception_residual_network_cross_validation'
    model = training(model=model,
                    model_name=model_name,
                    x_train=train_x,
                    y_train=train_y,
                    x_test=val_x,
                    y_test=val_y)
    index += 1
    test_score = model.evaluate(val_x, val_y, verbose=1)
    scores.append(test_score[1])
    print(test_score[1])
    model = None

Train on 2387 samples, validate on 599 samples
Epoch 1/100
2387/2387 [==============================] - 78s 33ms/step - loss: 2.4217 - acc: 0.3243 - val_loss: 2.1850 - val_acc: 0.4775
Epoch 2/100
2387/2387 [==============================] - 28s 12ms/step - loss: 0.6468 - acc: 0.8421 - val_loss: 2.1525 - val_acc: 0.5359
Epoch 3/100
2387/2387 [==============================] - 28s 12ms/step - loss: 0.3457 - acc: 0.9346 - val_loss: 14.5109 - val_acc: 0.0634
Epoch 4/100
2387/2387 [==============================] - 28s 12ms/step - loss: 0.3347 - acc: 0.9367 - val_loss: 2.2274 - val_acc: 0.5993
Epoch 5/100
2387/2387 [==============================] - 29s 12ms/step - loss: 0.2286 - acc: 0.9594 - val_loss: 3.3827 - val_acc: 0.6010
Epoch 6/100
2387/2387 [==============================] - 28s 12ms/step - loss: 0.3197 - acc: 0.9334 - val_loss: 15.4185 - val_acc: 0.0434
Epoch 7/100
2387/2387 [==============================] - 28s 12ms/step - loss: 0.3271 - acc: 0.9434 - val_loss: 6.9982 - val_acc:

2387/2387 [==============================] - 29s 12ms/step - loss: 0.0982 - acc: 0.9686 - val_loss: 0.5931 - val_acc: 0.8280
Epoch 6/100
2387/2387 [==============================] - 29s 12ms/step - loss: 0.0512 - acc: 0.9845 - val_loss: 0.5544 - val_acc: 0.8447
Epoch 7/100
2387/2387 [==============================] - 29s 12ms/step - loss: 0.0404 - acc: 0.9899 - val_loss: 0.4889 - val_acc: 0.8631
Epoch 8/100
2387/2387 [==============================] - 29s 12ms/step - loss: 0.0374 - acc: 0.9929 - val_loss: 0.6238 - val_acc: 0.8564
Epoch 9/100
2387/2387 [==============================] - 28s 12ms/step - loss: 0.0068 - acc: 0.9979 - val_loss: 0.3325 - val_acc: 0.9048
Epoch 10/100
2387/2387 [==============================] - 28s 12ms/step - loss: 0.0099 - acc: 0.9962 - val_loss: 0.4957 - val_acc: 0.8648
Epoch 11/100
2387/2387 [==============================] - 29s 12ms/step - loss: 0.0178 - acc: 0.9946 - val_loss: 0.4799 - val_acc: 0.8781
Epoch 12/100
2387/2387 [===========================

599/599 [==============================] - 2s 4ms/step
0.9415692810423187
Train on 2388 samples, validate on 598 samples
Epoch 1/100
2388/2388 [==============================] - 89s 37ms/step - loss: 2.2840 - acc: 0.3492 - val_loss: 1.9808 - val_acc: 0.3746
Epoch 2/100
2388/2388 [==============================] - 27s 11ms/step - loss: 0.6576 - acc: 0.8070 - val_loss: 1.1152 - val_acc: 0.7341
Epoch 3/100
2388/2388 [==============================] - 27s 11ms/step - loss: 0.2030 - acc: 0.9338 - val_loss: 0.8001 - val_acc: 0.7676
Epoch 4/100
2388/2388 [==============================] - 27s 11ms/step - loss: 0.1095 - acc: 0.9644 - val_loss: 0.6485 - val_acc: 0.8629
Epoch 5/100
2388/2388 [==============================] - 27s 11ms/step - loss: 0.0129 - acc: 0.9971 - val_loss: 0.5694 - val_acc: 0.8779
Epoch 10/100
2388/2388 [==============================] - 27s 11ms/step - loss: 0.0137 - acc: 0.9950 - val_loss: 0.6797 - val_acc: 0.8528
Epoch 11/100
2388/2388 [==============================] 

ResourceExhaustedError: OOM when allocating tensor with shape[100,2080,5,5] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: block8_2_conv_4/convolution = Conv2D[T=DT_FLOAT, _class=["loc:@train...propFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block8_2_mixed_4/concat, block8_2_conv_4/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: training_4/Adam/gradients/block8_6_4/add_grad/Shape/_97225 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_19146_training_4/Adam/gradients/block8_6_4/add_grad/Shape", tensor_type=DT_INT32, _device="/job:localhost

In [ ]:
print(scores)
print("Accuracy:%0.4f (+/- %0.4f)" %(np.array(scores).mean(),np.array(scores).std()*2))